In [4]:
from datetime import datetime
from fse_toolkit.utils.common import read_tif
from fse_toolkit.multispectral.scraper import fetch_weather_data
from fse_toolkit.multispectral.temperature import calculate_te1, calculate_te2

In [2]:
tif_files = [
    '../../../FPAR/To彥成/95183018_221211e_28~6584_hr4-003.tif',
    '../../../FPAR/To彥成/95183020_221120f_30~0012_hr4.tif',
    '../../../FPAR/To彥成/95183029_221120f_29~0113_hr4.tif',
    '../../../FPAR/To彥成/95183038_221211e_28~6592_hr4-019.tif',
    '../../../FPAR/To彥成/95183040_221120f_30~0004_hr4.tif',
    '../../../FPAR/To彥成/95183049_221120f_29~0121_hr4.tif'
]

In [3]:
tif_data = read_tif(tif_files[1])

影像的地理變換矩陣: | 0.25, 0.00, 222430.00|
| 0.00,-0.25, 2513820.00|
| 0.00, 0.00, 1.00|
影像的座標參考系統: EPSG:3826


In [5]:
date = tif_files[1].split('_')[1][:-1]
#parse date YYMMDD -> datetime
date = datetime.strptime(date, '%y%m%d')
print("date:", date)

start_ym = date.strftime('%Y-%m')
opt_ym = datetime(date.year, 6, 1).strftime('%Y-%m') # June of the same year
print("start_ym:", start_ym, "|", "opt_ym:", opt_ym)

date: 2022-11-20 00:00:00
start_ym: 2022-11 | opt_ym: 2022-06


In [6]:
if date.month < 6:
    weather_data = fetch_weather_data(
        tif_data['coordinates']['center'][0], tif_data['coordinates']['center'][1], 
        ['平均氣溫(℃)', '累積日射量(MJ/m2)'],
        start_ym, opt_ym
    )
else:
    weather_data = fetch_weather_data(
        tif_data['coordinates']['center'][0], tif_data['coordinates']['center'][1], 
        ['平均氣溫(℃)', '累積日射量(MJ/m2)'],
        opt_ym, start_ym
    )

/Users/azetry/Documents/Projects/2024/FSES/v2/fse_toolkit/src/fse_toolkit/data/station_record.csv
WebDriver setup completed. Temp dir: /Users/azetry/Documents/Projects/2024/FSES/v2/fse_toolkit/examples/temp/tmp0gpdk5pf


In [12]:
display(weather_data['data'])
display(weather_data['station_info'])

,測站代碼,平均氣溫(℃),累積日射量(MJ/m2)
觀測時間,,,
2022-06,72Q010,28.8,794.39
2022-07,72Q010,29.3,817.93
2022-08,72Q010,28.3,766.29
2022-09,72Q010,27.8,667.37
2022-10,72Q010,26.3,579.03
2022-11,72Q010,25.7,554.10


index                              55
station_id                     72Q010
area_name                          南部
station_name             高雄農改(72Q010)
station_type                      農業站
altitude(m)                      45.0
longitude                  120.525889
latitude                    22.709333
city                              屏東縣
address                 長治鄉德和村德和路2-6號
data_start_date            1931-07-01
station_closure_date              NaN
notes                             NaN
original_station_id               NaN
new_station_id                    NaN
Name: 0, dtype: object

In [13]:
sol = weather_data['data'].loc[start_ym, '累積日射量(MJ/m2)']
t = weather_data['data'].loc[start_ym, '平均氣溫(℃)']
t_opt = weather_data['data'].loc[opt_ym, '平均氣溫(℃)']
print("sol:", sol, "|", "t:", t, "|", "t_opt:", t_opt)

sol: 554.1 | t: 25.7 | t_opt: 28.8


In [14]:
te1 = calculate_te1(t_opt)
te2 = calculate_te2(t_opt, t)
print("te1:", te1, "|", "te2:", te2)

te1: 0.9612800000000001 | te2: 0.9257429758096202
